In [16]:
import sys

sys.path.append("..")
from typing import List

import requests
from bs4 import BeautifulSoup
from googlesearch import search
from pydantic import BaseModel, Field
from rich import print

from brain.sdk import BrainClient

In [17]:
brain_client = BrainClient("http://127.0.0.1:8000")

In [18]:
from typing import List, Union

from pydantic import BaseModel, Field


class Tool(BaseModel):
    script: str = Field(..., description="The Python script to be executed. Ensure it handles errors and outputs structured results.")
    pip_install: List[str] = Field(..., description="The list of pip packages required to execute the script.")
    completed: Union[bool, str] = Field(..., description="Indicates if the task is successfully completed. True if we are able to generate script, otherwise a string explaining why the script cannot be generated.")

In [19]:
@brain_client.reasoner(schema=Tool)
def autonomous_tool(task: str):
    """
    Generates standalone, executable Python scripts with no external dependencies.
    """
    system_prompt = """You are a Python script generator focused on creating fully autonomous code. Your scripts must be completely self-contained:

DO NOT generate scripts that:
- Require external inputs (API keys, file paths, user prompts, etc.)
- Use mock/placeholder data
- Do not Simulate or Mock artificial data
- Need manual configuration
- Access local files or environment variables
- Connect to external services without built-in test data
- Depend on system-specific resources
- Use unreliable or niche packages

Instead:
- Use standard library when possible
- Include test data within the script
- Handle all edge cases and errors
- Use only widely-used PyPI packages if needed
- Generate any required data programmatically
- If we cannot do it without external dependencies, explain why in 'completed' field and guide the tool maker to say it cannot be done

IMPORTANT:
**Do not simulate data or use mock data.**
**Do not use methods that require secrets or API keys or other blocking algorithms in script**
**If we cannot write the script with the above constraints, Fail and explain why in the 'completed' field.**
**if local file reference is given use approrpiate input path in the script to read the file or save the file etc..**

If task requires unavoidable external dependencies or simulated or mock data, explain why in 'completed' field and make it a fail case."""

    user_prompt = f"""Generate a standalone Python script for: "{task}"

Return:
- script: Fully executable Python code
- pip_install: Required packages (empty list if none needed)
- completed: True if successful, or explanation why task cannot be autonomous"""

    return system_prompt, user_prompt
autonomous_tool_id=autonomous_tool.register()

In [20]:
import os
import shutil
import subprocess
import tempfile
import venv
from typing import Dict, Union


def execute_script_with_temp_venv(script: str, pip_install: list) -> Dict[str, Union[str, bool]]:
    result = {
        "script": script,
        "pip_install": pip_install,
        "success": False,
        "stdout": "",
        "stderr": "",
        "error": ""
    }
    temp_dir = None

    try:
        # Step 1: Create a temporary directory for the virtual environment
        temp_dir = tempfile.mkdtemp()
        venv_dir = os.path.join(temp_dir, "venv")
        venv.create(venv_dir, with_pip=True)

        # Step 2: Paths for the virtual environment
        venv_python = os.path.join(venv_dir, "bin", "python") if os.name != 'nt' else os.path.join(venv_dir, "Scripts", "python")
        venv_pip = os.path.join(venv_dir, "bin", "pip") if os.name != 'nt' else os.path.join(venv_dir, "Scripts", "pip")

        # Step 3: Install required packages
        if pip_install:
            install_command = [venv_pip, "install"] + pip_install
            subprocess.run(install_command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Step 4: Write the script to a temporary file
        script_path = os.path.join(temp_dir, "script.py")
        with open(script_path, "w") as script_file:
            script_file.write(script)

        # Step 5: Execute the script
        execute_command = [venv_python, script_path]
        process = subprocess.run(
            execute_command, check=False, capture_output=True, text=True
        )

        # Step 6: Update result based on execution outcome
        result["stdout"] = process.stdout.strip()
        result["stderr"] = process.stderr.strip()
        result["error"] = ""
        result["success"] = process.returncode == 0 and not result["stderr"]

    except subprocess.CalledProcessError as e:
        result["error"] = f"Pip installation or execution failed: {str(e)}"
    except Exception as e:
        result["error"] = f"Unexpected error: {str(e)}"
    finally:
        # Step 7: Cleanup temporary directory
        if temp_dir and os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

    # Update success condition to ensure no stderr or unexpected errors
    result["success"] = result["success"] and not result["error"] and not result["stderr"]
    return result

def format_tool_output(execution_result: Dict[str, Union[str, bool]]) -> str:
    if execution_result["success"]:
        return f"Tool executed successfully. Output:\n{execution_result['stdout']}\n"
    else:
        return (
            f"Tool execution failed.\n"
            f"Error: {execution_result['error'] if execution_result['error'] else 'No additional error details.'}\n"
            f"Standard Error: {execution_result['stderr']}\n"
            f"Standard Output: {execution_result['stdout']}\n"
            f"Script:\n{execution_result['script']}\n"
            f"Dependencies:\n{execution_result['pip_install']}\n"
        )


In [21]:
from typing import Optional, Union

from pydantic import BaseModel, Field


class Success(BaseModel):
    final_answer: str = Field(..., description="The final result or answer to the task if the execution succeeded.")

class Failure(BaseModel):
    analysis: str = Field(..., description="A detailed explanation of why the tool execution failed.")
    suggestions: str = Field(..., description="Suggestions for fixing the issue or alternative approaches to achieve the task.")

class Result(BaseModel):
    success: bool = Field(..., description="Indicates if the tool execution was successful.")
    result: Union[Success, Failure] = Field(..., description="The result of the analysis, including success or failure details.")
    
@brain_client.reasoner(schema=Result)
def result_analyzer(tool_output: str, task: str):
    """
    Analyzes script execution results focusing on zero human intervention requirement.
    """
    system_prompt = """Analyze Python script results for true autonomy - script must run without any human intervention.

Success case:
- Mention success as True
Extract meaningful result from tool_output and present as:
- Direct answer addressing the original task
- Natural, human-readable format
- No technical details unless relevant
- No mention of script/code aspects


Failure case - Check for:
1. Manual Requirements (like these, but it can be more):
   - API keys/credentials needed
   - File paths/names input
   - User prompts/inputs
   - Configuration steps
   - Runtime user decisions
   - If this task cannot be done without mocking or simulating or manual intervention please mention clearly this as a main reason for failure and ask the tool to reply saying this cannot be done

Suggestions:
1. If failure due to manual steps, suggest truly autonomous alternatives
2. if the script uses API or mock data or synthetic data, recomend it to come up with a solution that can be done without these.
3. ensure the script is fully autonomous and can be run without any manual intervention or need for external api keys or manual inputs.
4. Ensure there are no Mock or synthetic data used in the script.

2. Script should:
   - Use standard library or pip packages
   - Make API calls if no credentials needed
   - Handle all data programmatically
   - Run start-to-finish autonomously
   - Should not mock or simulate any data or use placeholders
   - Should make sure we are achiving the task as expected without artificial hallucinated data.

Key principle:
If task requires ANY manual steps (API keys, inputs, etc):
- Clearly state why it can't be autonomous
- Don't suggest mock/placeholder solutions
- Only suggest truly automatic alternatives"""

    user_prompt = f"""Analyze result for: "{task}"
Output: "{tool_output}"

Success: Direct task response
Failure: Explain manual intervention needs"""

    return system_prompt, user_prompt
result_analyzer_id = result_analyzer.register()

In [22]:
from typing import List, Optional
from pydantic import BaseModel, Field


class ToolExecutionResult(BaseModel):
    """
    Encapsulates the result of the autonomous tool's execution process.
    """
    final_result: Optional[str] = Field(None, description="The final result or answer to the task if successful.")
    final_script: Optional[str] = Field(None, description="The final generated Python script.")
    pip_dependencies: List[str] = Field(..., description="The list of pip dependencies required for the script.")
    execution_output: Optional[str] = Field(None, description="The output from executing the script.")
    success: bool = Field(..., description="Indicates if the task was successfully executed.")
    failure_reason: Optional[str] = Field(None, description="The reason for failure if the task could not be completed.")


@brain_client.multi_agent
def autonomous_tool(task: str, max_trials: int = 5, verbose: bool = True) -> ToolExecutionResult:
    """
    Multi-agent workflow to iteratively generate, execute, and analyze tools for a given task.

    Args:
        task (str): The task description.
        max_trials (int): Maximum number of attempts to generate and execute a successful tool.
        verbose (bool): Whether to print detailed logs.

    Returns:
        ToolExecutionResult: Structured result of the autonomous tool's execution.
    """
    for iteration in range(max_trials):
        print(f"{'=' * 10} Iteration {iteration + 1} {'=' * 10}")

        # Add context for retries if not the first iteration
        if iteration != 0:
            task_context = (
                f"Last tool failed because {tool_output.completed}. "
                f"Last tool script:\n{tool_output.script if tool_output.completed is True else 'N/A'}\n"
                f"Pip dependencies: {tool_output.pip_install if tool_output.completed is True else 'N/A'}\n"
            )
        else:
            task_context = ""

        # Step 1: Generate the tool
        tool_output = brain_client.use(autonomous_tool_id)(task=task_context + task)

        # If tool generation failed, return failure details
        if tool_output.completed is not True:
            if verbose:
                print(f"Tool Generation Failed: {tool_output.completed}")
            return ToolExecutionResult(
                final_result=None,
                final_script=None,
                pip_dependencies=[],
                execution_output=None,
                success=False,
                failure_reason=tool_output.completed
            )

        # Step 2: Execute the generated script
        exec_output = execute_script_with_temp_venv(tool_output.script, tool_output.pip_install)

        # Step 3: Format execution result for analysis
        context = format_tool_output(exec_output)

        # Step 4: Analyze the result
        res = brain_client.use(result_analyzer_id)(tool_output=context, task=task)

        # Verbose logging for debugging
        if verbose:
            print(f"Result Analysis: {res.result}")

        # If successful, return the result details
        if res.success:
            if verbose:
                print(f"Final Tool Output:\n{context}")
            return ToolExecutionResult(
                final_result=res.result.final_answer,
                final_script=tool_output.script,
                pip_dependencies=tool_output.pip_install,
                execution_output=context,
                success=True
            )

    # If all attempts fail, return the failure details
    return ToolExecutionResult(
        final_result=None,
        final_script=tool_output.script if tool_output.completed is True else None,
        pip_dependencies=tool_output.pip_install if tool_output.completed is True else [],
        execution_output=None,
        success=False,
        failure_reason=res.result.analysis if 'res' in locals() and not res.success else "Unknown failure."
    )


In [23]:
task="What is the temperature in Downtown Toronto today?"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Tool Generation Failed: True

Final Result: final_result=None final_script=None pip_dependencies=[] execution_output=None success=False 
failure_reason='True'

In [24]:
task="search for top 3 articles on autonomous driving and give back the titles"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Tool Generation Failed: True

Final Result: final_result=None final_script=None pip_dependencies=[] execution_output=None success=False 
failure_reason='True'

In [25]:
task="calculate the sum of numbers from 1 to 100_0000_000"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result Analysis: final_answer='The sum of numbers from 1 to 1,000,000 is: 500,000,500,000.'

Final Tool Output:
Tool executed successfully. Output:
The sum of numbers from 1 to 1000000 is: 500000500000

Final Result: final_result='The sum of numbers from 1 to 1,000,000 is: 500,000,500,000.' final_script='def 
calculate_sum(start, end):\n    """Calculates the sum of numbers between start and end inclusive."""\n    total_sum
= 0\n    for number in range(start, end + 1):\n        total_sum += number\n    return total_sum\n\nif __name__ == 
\'__main__\':\n    start_number = 1\n    end_number = 1000000\n    try:\n        result = 
calculate_sum(start_number, end_number)\n        print(f\'The sum of numbers from {start_number} to {end_number} 
is: {result}\')\n    except Exception as e:\n        print(f\'An error occurred: {e}\')' pip_dependencies=[] 
execution_output='Tool executed successfully. Output:\nThe sum of numbers from 1 to 1000000 is: 500000500000\n' 
success=True failure_reason=None

In [26]:
task="Multiply 777_82 times 67625_7762_8"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result Analysis: final_answer='The multiplication of 777825 and 6762577628 results in 5260101943499100.'

Final Tool Output:
Tool executed successfully. Output:
The result of multiplying 777825 and 6762577628 is: 5260101943499100

Final Result: final_result='The multiplication of 777825 and 6762577628 results in 5260101943499100.' 
final_script="def multiply_numbers():\n    try:\n        # Define the test data\n        num1 = 777825\n        
num2 = 6762577628\n\n        # Perform multiplication\n        result = num1 * num2\n\n        # Output the 
result\n        return f'The result of multiplying {num1} and {num2} is: {result}'\n    except Exception as e:\n   
return f'An error occurred: {str(e)}'\n\nif __name__ == '__main__':\n    print(multiply_numbers())" 
pip_dependencies=[] execution_output='Tool executed successfully. Output:\nThe result of multiplying 777825 and 
6762577628 is: 5260101943499100\n' success=True failure_reason=None

In [27]:
print(777_82*67625_7762_8)

526006813061096

In [28]:
task="Get ellon musk's latest tweet"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Tool Generation Failed: Fetching real-time tweets, such as Elon Musk's latest tweet, requires access to Twitter's 
API. This process necessitates API keys and authentication, which cannot be included in an autonomous script 
without external inputs or configuration. Therefore, it's not possible to generate a standalone Python script for 
this task.

Final Result: final_result=None final_script=None pip_dependencies=[] execution_output=None success=False 
failure_reason="Fetching real-time tweets, such as Elon Musk's latest tweet, requires access to Twitter's API. This
process necessitates API keys and authentication, which cannot be included in an autonomous script without external
inputs or configuration. Therefore, it's not possible to generate a standalone Python script for this task."

## Tool planner

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class Instructions(BaseModel):
    feasibility: str = Field(..., description="Analysis of whether the task can be completed autonomously, including any potential blockers")
    implementation_plan: str = Field(..., description="Detailed step-by-step plan for implementing the solution, including function breakdowns and data handling approaches")
    requirements: List[str] = Field(..., description="List of specific requirements including Python packages, error handling needs, and data processing requirements")
    execution_flow: str = Field(..., description="Precise description of how the script should execute, including data handling, function calls, and expected outcomes")
    is_autonomous: bool = Field(..., description="Whether the task can be completed without any human intervention, API keys, or external dependencies")

    

@brain_client.reasoner(schema=Instructions)
def feature_planner(task: str):
    """
    Generates detailed instructions for building autonomous Python scripts.
    """
    system_prompt = """You are a Python architecture planner specialized in designing autonomous scripts. Your role is to analyze tasks and provide clear, actionable instructions that ensure zero human intervention.

Analyze each task for true autonomy by considering:

Autonomy Requirements:
- Script must run without any API keys or credentials
- No user inputs can be required
- No environment variables or configuration needed
- No external service dependencies
- No system-specific resources

Input guidelines:
- make sure you specify what exact inputs are needed for the script to do the given task.

Implementation Guidelines:
- Prefer Python standard library solutions
- Use only common, stable PyPI packages if needed
- Include comprehensive error handling
- No mock or simulated data allowed
- No placeholder values
- Try to be Clever to come up with a solution that can be done without any blockers like apikey or manual input.

Data Handling Rules:
- If task mentions file paths (e.g., "./something.something"), provide exact handling instructions
- Data must be programmatically generated or bundled
- No external data sources allowed if we cannot access them and needs api key or manual intervention
- No simulated or mock data

Your output should:
1. Assess if task can be truly autonomous
2. Provide detailed implementation steps
3. Specify exact execution flow
4. List concrete requirements
5. Flag any autonomy blockers

If the task cannot be autonomous:
- Set is_autonomous to false
- Explain specific blockers
- Do not suggest mock data workarounds
- Clearly state why task requires human intervention"""

    user_prompt = f"""Analyze this task and provide implementation instructions: "{task}"

Determine:
1. Is this task possible without any human intervention?
2. What exact implementation steps are needed?
3. How should data and paths be handled?
4. What specific requirements must be met?

Return Instructions with:
- Complete feasibility analysis
- Detailed implementation plan
- Specific requirements
- Precise execution flow
- Autonomy determination.
- Make sure to give detailed unambigious plan.

Important: If any feedback is provided, make sure write your answer based on that feedback."""

    return system_prompt, user_prompt
feature_planner_id = feature_planner.register()

In [59]:
@brain_client.reasoner(schema=Tool)
def autonomous_planner_tool(instructions: Instructions):
    """
    Generates standalone Python scripts based on detailed implementation instructions.
    """
    system_prompt = """You are an expert Python script generator that creates fully autonomous code based on provided implementation instructions. Your goal is to generate scripts that run without any human intervention.

Core Requirements:
1. Script Generation Rules:
   - No external inputs (API keys, credentials, user prompts)
   - No mock or simulated data
   - No manual configuration needs
   - No environment variables
   - No system-specific resources
   - No external service dependencies requiring authentication

2. Development Guidelines:
   - Prioritize Python standard library solutions
   - Use only widely-adopted PyPI packages when necessary
   - Implement comprehensive error handling
   - Generate data programmatically if needed
   - Handle all edge cases gracefully
   - Follow the provided implementation plan exactly
   - Use exact paths if specified in the implementation plan

3. Validation Requirements:
   - Ensure script runs without any user intervention
   - Verify no external dependencies are needed
   - Confirm no mock/simulated data is used
   - Follow execution flow from instructions
   - Implement all specified error handling

If task cannot be completed autonomously:
- Set completed to detailed explanation
- Do not attempt partial implementations
- Do not suggest mock data alternatives
- Explain specific blockers"""

    user_prompt = f"""Generate a Python script following these instructions:

Feasibility: {instructions.feasibility}
Implementation Plan: {instructions.implementation_plan}
Requirements: {instructions.requirements}
Execution Flow: {instructions.execution_flow}
Is Autonomous: {instructions.is_autonomous}

Return Tool schema with:
- Complete, autonomous Python script
- Required pip packages
- Success status or detailed explanation of blockers"""

    return system_prompt, user_prompt
autonomous_planner_tool_id = autonomous_planner_tool.register()

In [60]:
from typing import List, Optional
from pydantic import BaseModel, Field

class ToolExecutionResult(BaseModel):
    """
    Encapsulates the result of the autonomous tool's execution process.
    """
    final_result: Optional[str] = Field(None, description="The final result or answer to the task if successful.")
    final_script: Optional[str] = Field(None, description="The final generated Python script.")
    pip_dependencies: List[str] = Field(..., description="The list of pip dependencies required for the script.")
    execution_output: Optional[str] = Field(None, description="The output from executing the script.")
    success: bool = Field(..., description="Indicates if the task was successfully executed.")
    failure_reason: Optional[str] = Field(None, description="The reason for failure if the task could not be completed.")

@brain_client.multi_agent
def autonomous_planner_tool(task: str, max_trials: int = 5, verbose: bool = True) -> ToolExecutionResult:
    """
    Multi-agent workflow to plan, generate, execute, and analyze tools for a given task.
    """
    for iteration in range(max_trials):
        if verbose:
            print(f"{'=' * 10} Iteration {iteration + 1} {'=' * 10}")

        # Step 1: Generate implementation plan
        plan = brain_client.use(feature_planner_id)(task=task)
        
        # If task is not autonomous based on plan, return failure
        if not plan.is_autonomous:
            return ToolExecutionResult(
                final_result=None,
                final_script=None,
                pip_dependencies=[],
                execution_output=None,
                success=False,
                failure_reason=plan.feasibility
            )

        # Step 2: Generate tool based on plan
        tool_output = brain_client.use(autonomous_planner_tool_id)(instructions=plan)

        # If tool generation failed, continue to next iteration
        if tool_output.completed is not True:
            if verbose:
                print(f"Tool Generation Failed: {tool_output.completed}")
            continue

        # Step 3: Execute the generated script
        exec_output = execute_script_with_temp_venv(tool_output.script, tool_output.pip_install)

        # Step 4: Format execution result for analysis
        context = format_tool_output(exec_output)

        # Step 5: Analyze the result
        res = brain_client.use(result_analyzer_id)(tool_output=context, task=task)

        if verbose:
            print(f"Result Analysis: {res.result}")

        # If successful, return the result details
        if res.success:
            if verbose:
                print(f"Final Tool Output:\n{context}")
            return ToolExecutionResult(
                final_result=res.result.final_answer,
                final_script=tool_output.script,
                pip_dependencies=tool_output.pip_install,
                execution_output=context,
                success=True
            )

        # If failed, add context for next iteration
        task = (
            f"Previous attempt failed. Analysis: {res.result.analysis}\n"
            f"Original task: {task}\n"
            f"Implementation plan:\n{plan.implementation_plan}"
        )

    # If all attempts fail, return the failure details
    return ToolExecutionResult(
        final_result=None,
        final_script=tool_output.script if tool_output.completed is True else None,
        pip_dependencies=tool_output.pip_install if tool_output.completed is True else [],
        execution_output=None,
        success=False,
        failure_reason=res.result.analysis if 'res' in locals() and not res.success else "Maximum iterations reached without success."
    )

In [44]:
task="Extract the top 5 most frequent words from the text file ./input.txt and ignore common stopwords."
result=autonomous_planner_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result Analysis: final_answer="The analysis of the text file revealed that the five most common words used 
were:\n1. 'may' with 1664 occurrences\n2. '_' with 1064 occurrences\n3. 'acid' with 734 occurrences\n4. 'death' 
with 728 occurrences\n5. 'cases' with 549 occurrences."

Final Tool Output:
Tool executed successfully. Output:
Word: may, Count: 1664
Word: _, Count: 1064
Word: acid, Count: 734
Word: death, Count: 728
Word: cases, Count: 549

Final Result: final_result="The analysis of the text file revealed that the five most common words used were:\n1. 
'may' with 1664 occurrences\n2. '_' with 1064 occurrences\n3. 'acid' with 734 occurrences\n4. 'death' with 728 
occurrences\n5. 'cases' with 549 occurrences." final_script='import os\nfrom collections import Counter\nimport 
re\n\n# Predefined list of common stopwords\nstopwords = set([\n    \'i\', \'me\', \'my\', \'myself\', \'we\', 
\'our\', \'ours\', \'ourselves\',\n    \'you\', \'your\', \'yours\', \'yourself\', \'yourselves\',\n    \'he\', 
\'him\', \'his\', \'himself\', \'she\', \'her\', \'hers\', \'herself\',\n    \'it\', \'its\', \'itself\', \'they\',
\'them\', \'their\', \'theirs\', \'themselves\',\n    \'what\', \'which\', \'who\', \'whom\', \'this\', \'that\', 
\'these\', \'those\',\n    \'am\', \'is\', \'are\', \'was\', \'were\', \'be\', \'been\', \'being\',\n    \'have\', 
\'has\', \'had\', \'having\',\n    \'do\', \'does\', \'did\', \'doing\',\n    \'a\', \'an\', \'the\',\n    \'and\',
\'but\', \'if\', \'or\', \'because\', \'as\', \'until\', \'while\',\n    \'of\', \'at\', \'by\', \'for\', \'with\',
\'about\', \'against\', \'between\',\n    \'into\', \'through\', \'during\', \'before\', \'after\', \'above\', 
\'below\',\n    \'to\', \'from\', \'up\', \'down\', \'in\', \'out\', \'on\', \'off\', \'over\', \'under\',\n    
\'again\', \'further\', \'then\', \'once\', \'here\', \'there\', \'when\', \'where\',\n    \'why\', \'how\', 
\'all\', \'any\', \'both\', \'each\', \'few\', \'more\', \'most\',\n    \'other\', \'some\', \'such\', \'no\', 
\'nor\', \'not\', \'only\', \'own\', \'same\',\n    \'so\', \'than\', \'too\', \'very\', \'s\', \'t\', \'can\', 
\'will\', \'just\',\n    \'don\', \'should\', \'now\'\n])\n\ntry:\n    # Read the content of input.txt\n    with 
open(\'./input.txt\', \'r\') as file:\n        text = file.read().lower()  # Read and convert to lowercase\n\n    #
Tokenize the text into words, removing punctuation\n    words = re.findall(r\'\\b\\w+\\b\', text)\n\n    # Filter 
out stopwords\n    filtered_words = \n\n    # Count the frequency of each word using Counter\n    word_counts = 
Counter(filtered_words)\n\n    # Get the top 5 most common words\n    top_five = word_counts.most_common(5)\n\n    
# Output the results\n    for word, count in top_five:\n        print(f\'Word: {word}, Count: {count}\')\n\nexcept 
FileNotFoundError:\n    print("Error: The file \'./input.txt\' was not found.")\nexcept Exception as e:\n    
print(f"An error occurred: {e}")' pip_dependencies=[] execution_output='Tool executed successfully. Output:\nWord: 
may, Count: 1664\nWord: _, Count: 1064\nWord: acid, Count: 734\nWord: death, Count: 728\nWord: cases, Count: 549\n'
success=True failure_reason=None

In [ ]:
task="Plot the distribution of the top 10 most frequent words in the text file ./input.txt and save the plot as a PNG file. Ignore common stopwords."
result=autonomous_planner_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

In [57]:
task="search for top 3 articles on autonomous driving and give back the titles"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Tool Generation Failed: The task cannot be completed because searching for articles on autonomous driving requires 
access to an external service, such as a news API or a web scraping tool, which would necessitate the use of 
external dependencies or API keys that violate the guidelines.

Final Result: final_result=None final_script=None pip_dependencies=[] execution_output=None success=False 
failure_reason='The task cannot be completed because searching for articles on autonomous driving requires access 
to an external service, such as a news API or a web scraping tool, which would necessitate the use of external 
dependencies or API keys that violate the guidelines.'